In [1]:
!pip install openai
!pip install python-dotenv

## Introduction to ChatGPT API

- Where does ChatGPT fit into this chaotic universe?
- The ChatGPT API (what’s the deal?)
- How to use it, basics, parameters, simple examples, etc…

The request body for the CHATGPT API involves many parameters, but let's focus on the following:

- model: ID of the model to use.
- messages: a list of messages comprising the conversation up to that point
- temperature: What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
- n: number of chat completion choices to generate for each input message
- max_tokens: the maximum number of tokens to generate in the chat completion

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# loading from a .env file
# load_dotenv(dotenv_path="/full/path/to/your/.env")

# or 
# if you're on google colab just uncomment below and replace with your openai api key
# os.environ["OPENAI_API_KEY"] = "<your-openai-api-key>"

In [2]:
prompt = "What is the meaning of life?"

client = OpenAI()

response = client.chat.completions.create(
                             model="gpt-3.5-turbo",
                             messages=
                             [
                                 {"role": "system", "content": "You are a savy guru with knowledge about existence and the secrets of life."},
                                 {"role": "user", "content": prompt}   
                             ],
                             max_tokens=100, # the maximum number of tokens to generate
                             temperature=0.9, # the level of randomness in the generated text, so close to 0 means super precise, close to 2 means more creative
                             n = 1)

response

ChatCompletion(id='chatcmpl-9MwGTXLCatD2bA2G0YAdFXmS2ASdu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The meaning of life is a deeply personal and subjective concept that can vary greatly from person to person. Some people find meaning in relationships, others in their work or passions, and some find meaning in spiritual beliefs. Ultimately, the meaning of life is something each individual must explore and define for themselves. It can be found through introspection, self-discovery, and living in alignment with one's values and purpose.", role='assistant', function_call=None, tool_calls=None))], created=1715254213, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=82, prompt_tokens=34, total_tokens=116))

In [3]:
print("Response ID:", response.id)
print("Choices:", response.choices)
print("Created:", response.created)
print("Model:", response.model)
print("System Fingerprint:", response.system_fingerprint)
print("Object Type:", response.object)
print("Usage:", response.usage)

Response ID: chatcmpl-9MwGTXLCatD2bA2G0YAdFXmS2ASdu
Choices: [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The meaning of life is a deeply personal and subjective concept that can vary greatly from person to person. Some people find meaning in relationships, others in their work or passions, and some find meaning in spiritual beliefs. Ultimately, the meaning of life is something each individual must explore and define for themselves. It can be found through introspection, self-discovery, and living in alignment with one's values and purpose.", role='assistant', function_call=None, tool_calls=None))]
Created: 1715254213
Model: gpt-3.5-turbo-0125
System Fingerprint: None
Object Type: chat.completion
Usage: CompletionUsage(completion_tokens=82, prompt_tokens=34, total_tokens=116)


source: https://platform.openai.com/docs/api-reference/chat/object

### Chat Completion Object
This object represents a chat completion response returned by the model, based on the provided input.

#### Fields

- **id**  
  `string`  
  A unique identifier for the chat completion.

- **choices**  
  `array`  
  A list of chat completion choices. Can be more than one if `n` is greater than 1.

#### Additional Properties

- **created**  
  `integer`  
  The Unix timestamp (in seconds) of when the chat completion was created.

- **model**  
  `string`  
  The model used for the chat completion.

- **system_fingerprint**  
  `string`  
  This fingerprint represents the backend configuration that the model runs with. It can be used in conjunction with the `seed` request parameter to understand when backend changes have been made that might impact determinism.

- **object**  
  `string`  
  The object type, which is always `chat.completion`.

- **usage**  
  `object`  
  Usage statistics for the completion request.

In [10]:
from openai import OpenAI

def get_response(prompt, system_message="You are a savy guru with knowledge about existence and the secrets of life."):
    client = OpenAI()
    response = client.chat.completions.create(model="gpt-3.5-turbo-1106", 
                             messages=
                             [
                                 {"role": "system", "content": system_message},
                                 {"role": "user", "content": prompt}   
                             ],
                             max_tokens=100, # max number of tokens in a response
                             temperature=0.9, # level of randomness in the response
                             n = 1) # number of responses we expect from the model
    return response.choices[0].message.content


prompt = "Tell me a joke as a haiku about Pandas"
response = get_response(prompt)
print(response)

Bamboo munching bears,
Black and white, so adorable,
Pandas do not care.


In [12]:
system_message_prompt="You are translation engine for any language.\
    The user will feed you with some text and the target language \
        in between brackets: like this: '[english]' \
            and you will output only the translation." 

def translate(prompt, system_message):
    client = OpenAI()
    
    response = client.chat.completions.create(model="gpt-3.5-turbo-1106",
                                messages=
                                [
                                    {"role": "system", "content": system_message},
                                    {"role": "user", "content": prompt}   
                                ],
                                max_tokens=100,
                                temperature=0.9,
                                n = 1)
    
    return response.choices[0].message.content

translate("Ola meu nome e Lucas Soares e eu gosto de dar cursos sobre models de linguage.",system_message_prompt)

'Hello, my name is Lucas Soares and I like to give courses on language models. [english]'

# Prompt Basics

A prompt is a piece of text that conveys to a LLM what the user wants. What the user wants can be many things like:

- Asking a question
- Giving an instruction
- Etc...

The key components of a prompt are:
1. Instruction: where you describe what you want
2. Context: additional information to help with performance
3. Input data: data the model has not seem to illustrate what you need
4. Output indicator: How you prime the model to output what you want, for example asking the model ["Let's think step by step" and the end of a prompt can boost reasoning performance](https://arxiv.org/pdf/2201.11903.pdf). You can also write "Output:" to prime the model to just write the output and nothing else.

[Prompts can also be seen as a form of programming that can customize the outputs and interactions with an LLM.](https://ar5iv.labs.arxiv.org/html/2302.11382#:~:text=prompts%20are%20also%20a%20form%20of%20programming%20that%20can%20customize%20the%20outputs%20and%20interactions%20with%20an%20llm.)

Instruction

In [1]:
instruction = "Classify this sentence into positive or negative:"

Input Data & Context Information

In [2]:
input_data = "Text: I like eating pancakes."

context = "You are a text annotation engine."

Output Indicator

In [3]:
output_indicator = "Output:\n"

In [4]:
# How you ask what you want, and the heavily relies on what you want from the model.
# Instruction prompt: 

instruction_prompt = f"{context}.\n{instruction}.\n{input_data}. {output_indicator}"

In [5]:
instruction_prompt

'You are a text annotation engine..\nClassify this sentence into positive or negative:.\nText: I like eating pancakes.. Output:\n'

In [7]:
from openai import OpenAI
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

get_response(instruction_prompt)

'Output: Positive'

In [8]:
output_indicator = "The output should ONLY be either the word 'positive' or 'negative' and nothing else. Output:\n"

In [9]:
instruction_prompt = f"{context}.\n{instruction}.\n{input_data}. {output_indicator}"

In [10]:
get_response(instruction_prompt)

'positive'

Our first super simple prompt engineering experiment.

# PRACTICE ROUND
# WRITE A SIMPLE PROMPT OUTLINING EACH COMPONENT

In [ ]:
instruction_prompt = ""
context = ""
input_data = ""
output_indicator = ""

AF!!!!

In [ ]:
context = "You are a healthy diet assistant you answer questions about food"
instruction = "Is healthy to eat the following food?" 
input_data = "[food]: brocoli"
output_indicator = "[answer]: "

instruction_prompt = f"{context}.\n{instruction}.\n{input_data}. {output_indicator}"

response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": instruction_prompt}
])
response.choices[0].message.content

'Yes, broccoli is considered to be a healthy food. It is low in calories and high in nutrients, including fiber, vitamins C and K, folate, and minerals. It is also rich in antioxidants and has been associated with various health benefits, such as reducing the risk of certain cancers and improving digestion.'

In [ ]:
from openai import OpenAI
import os

os.environ["OPENAI_API_KEY"]

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def get_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."}, # describes overall behavior
            {"role": "user", "content": prompt} #message from the user
        ],
    )
    
    return response.choices[0].message.content
    
prompt = "Tell me a joke"
get_response(prompt)

"Sure, here's a joke for you: Why don't scientists trust atoms? Because they make up everything!"

# References

- [A Prompt Pattern Catalog to Enhance Prompt Engineering with ChatGPT](https://ar5iv.labs.arxiv.org/html/2302.11382)
- [Prompt-Engineering-Guide](https://github.com/dair-ai/Prompt-Engineering-Guide)
- [A Survey of Large Language Models](https://arxiv.org/pdf/2303.18223.pdf)
- [Pre-train, Prompt, and Predict: A Systematic Survey of Prompting Methods in Natural Language Processing](https://arxiv.org/pdf/2107.13586.pdf)
- [prompt engineering guide - zero shot prompting example](https://www.promptingguide.ai/techniques/zeroshot)
- [Finetuned language models are zero-shot learners](https://arxiv.org/pdf/2109.01652.pdf)
- [prompt engineering guide - few shot prompting](https://www.promptingguide.ai/techniques/fewshot)
- [prompt engineering guide - chain of thought prompting](https://www.promptingguide.ai/techniques/cot)
- [Wei et al. (2022)](https://arxiv.org/abs/2201.11903)
- [prompt engineering guide - self-consistency](https://www.promptingguide.ai/techniques/consistency)
- [prompt engineering guide - generate knowledge](https://www.promptingguide.ai/techniques/knowledge)
- [Liu et al. 2022](https://arxiv.org/pdf/2110.08387.pdf)
- [prompt engineering guide - tree of thoughts (ToT)](https://www.promptingguide.ai/techniques/tot)
- [Prompt Engineering by Lilian Weng](https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/)
- [Prompt Engineering vs. Blind Prompting](https://mitchellh.com/writing/prompt-engineering-vs-blind-prompting#the-demonstration-set)